In [ ]:
# this is scratch code I've used to load MPII.mat database.
# don't expect it will be goog/working.
# for working version see mpii_hdf5.py

In [245]:
from pycocotools.coco import COCO
from scipy.spatial.distance import cdist
import numpy as np
import cv2
import os
import os.path
import h5py
import json

from zipfile import ZipFile
from scipy.io import loadmat
from pprint import pprint


In [4]:
dataset_dir = os.path.abspath(os.path.join('..', 'dataset'))
mpii_dir = os.path.join(dataset_dir, "mpii/mpii_human_pose_v1")
keypoints_file = os.path.join(mpii_dir, "mpii_human_pose_v1_u12_1.mat")
images_dir = os.path.join(mpii_dir, "images")

tr_hdf5_path = os.path.join(dataset_dir, "mpii_train_dataset.h5")
val_hdf5_path = os.path.join(dataset_dir, "mpii_val_dataset.h5")


In [76]:
keypoints = loadmat(keypoints_file, struct_as_record=False )
keypoints = keypoints['RELEASE']
keypoints = keypoints[0,0]


In [280]:
def load_anno(keypoints, n):
    
    # crazy format :-/
    
    annolist = keypoints.annolist[0,n]

    assert  annolist.image.shape==(1,1)
    assert  annolist.image[0,0].name.shape==(1,)
    image = annolist.image[0,0].name[0]

    annorects = []
    
    assert annolist.annorect.shape[0]==1
    for i in range(annolist.annorect.shape[1]):
        assert           annolist.annorect[0,i].scale.shape==(1,1)
        annorect_scale = annolist.annorect[0,i].scale[0,0]
        assert              annolist.annorect[0,i].objpos.shape==(1,1)
        assert              annolist.annorect[0,i].objpos[0,0].x.shape==(1,1)
        annorect_objpos_x = annolist.annorect[0,i].objpos[0,0].x[0,0]
        assert              annolist.annorect[0,i].objpos[0,0].y.shape==(1,1)
        annorect_objpos_y = annolist.annorect[0,i].objpos[0,0].y[0,0]
        
        annopoints = []
        
        if 'annopoints' in vars(annolist.annorect[0,i]):
            assert annolist.annorect[0,i].annopoints.shape==(1,1)
            points = annolist.annorect[0,i].annopoints[0,0].point
            
            assert points.shape[0]==1
            for p in range(points.shape[1]):
                point = points[0,p]
                point._fieldnames==['id', 'x', 'y', 'is_visible']
                
                assert     point.id.shape==(1,1)
                point_id = point.id[0,0]
                
                assert    point.x.shape==(1,1)
                point_x = point.x[0,0]
                
                assert    point.y.shape==(1,1)
                point_y = point.y[0,0]
                
                assert             point.is_visible.shape==(1,1) or point.is_visible.shape==(0,0) or point.is_visible.shape==(1,), point.is_visible.shape
                point_is_visible = point.is_visible[0,0] if point.is_visible.shape==(1,1) else point.is_visible[0] if point.is_visible.shape==(1,) else None
                
                annopoints.append( { 'id':point_id, 'x':point_x, 'y':point_y, 'is_visible':point_is_visible } )
        
        head = None
        
        if 'x1' in vars(annolist.annorect[0,i]):
            assert ( annolist.annorect[0,i].x1.shape, annolist.annorect[0,i].x2.shape, annolist.annorect[0,i].y1.shape, annolist.annorect[0,i].y2.shape )  == ((1,1),(1,1),(1,1),(1,1))
            head = { 'x1':annolist.annorect[0,i].x1[0,0], 'x2':annolist.annorect[0,i].x2[0,0], 'y1':annolist.annorect[0,i].y1[0,0], 'y2':annolist.annorect[0,i].y2[0,0]  }
        
        annorects.append( { 'scale':annorect_scale, 'objpos':{'x':annorect_objpos_x, 'y':annorect_objpos_y}, 'head':head, 'annopoints':annopoints if len(annopoints)>0 else None } )
    
    assert      annolist.frame_sec.shape==(1,0) or annolist.frame_sec.shape==(1,1)
    frame_sec = annolist.frame_sec[0,0]         if annolist.frame_sec.shape==(1,1) else None
    
    assert   annolist.vididx.shape==(1,0) or annolist.vididx.shape==(1,1)
    vididx = annolist.vididx[0,0]         if annolist.vididx.shape==(1,1) else None
    
    assert      keypoints.img_train.shape[0]==1
    img_train = keypoints.img_train[0,n]

    assert    keypoints.version.shape==(1,)
    version = keypoints.version[0]
    
    single_person = []
    assert         keypoints.single_person[n,0].shape[1]==1
    for i in range(keypoints.single_person[n,0].shape[0]):
        single_person.append(keypoints.single_person[n,0][i,0])
    
    assert   keypoints.act[n,0].act_id.shape==(1,1)
    act_id = keypoints.act[n,0].act_id[0,0]
    
    assert     keypoints.act[n,0].act_name.shape==(0,) or keypoints.act[n,0].act_name.shape==(1,) 
    act_name = keypoints.act[n,0].act_name[0]          if keypoints.act[n,0].act_name.shape==(1,) else None
    
    assert     keypoints.act[n,0].cat_name.shape==(0,) or keypoints.act[n,0].cat_name.shape==(1,) 
    cat_name = keypoints.act[n,0].cat_name[0]     if keypoints.act[n,0].cat_name.shape==(1,) else None
    
    video_name = None
    if vididx is not None:
        video_name = "https://www.youtube.com/watch?v=" + keypoints.video_list[0,vididx-1][0] 
    
    
    return { 'image':image, 'annorects':annorects, 'img_train':img_train, 'version':version, 'single_person':single_person, 'act':{'act_id':act_id, 'act_name':act_name, 'cat_name':cat_name}, 'video_name':video_name, 'vididx':vididx, 'frame_sec':frame_sec } 
    
    


In [281]:
for n in range(10):
    pprint(load_anno(keypoints,n))
    print()

{'act': {'act_id': -1, 'act_name': None, 'cat_name': None},
 'annorects': [{'annopoints': None,
                'head': None,
                'objpos': {'x': 601, 'y': 380},
                'scale': 3.8807339512004684}],
 'frame_sec': None,
 'image': '037454012.jpg',
 'img_train': 0,
 'single_person': [1],
 'version': '12',
 'video_name': None,
 'vididx': None}

{'act': {'act_id': -1, 'act_name': None, 'cat_name': None},
 'annorects': [{'annopoints': None,
                'head': None,
                'objpos': {'x': 881, 'y': 394},
                'scale': 8.0781661285219926},
               {'annopoints': None,
                'head': None,
                'objpos': {'x': 338, 'y': 210},
                'scale': 8.9041293791139395}],
 'frame_sec': None,
 'image': '095071431.jpg',
 'img_train': 0,
 'single_person': [],
 'version': '12',
 'video_name': None,
 'vididx': None}

{'act': {'act_id': -1, 'act_name': None, 'cat_name': None},
 'annorects': [{'annopoints': None,
               

In [282]:
pprint(load_anno(keypoints,18099))



{'act': {'act_id': 487,
         'act_name': 'sitting, arts and crafts,  carving wood, weaving, spinning wool',
         'cat_name': 'miscellaneous'},
 'annorects': [{'annopoints': [{'id': 0,
                                'is_visible': '1',
                                'x': 435,
                                'y': 1066},
                               {'id': 1, 'is_visible': '1', 'x': 181, 'y': 811},
                               {'id': 2, 'is_visible': '1', 'x': 668, 'y': 845},
                               {'id': 3, 'is_visible': '1', 'x': 879, 'y': 879},
                               {'id': 4,
                                'is_visible': '1',
                                'x': 649,
                                'y': 1003},
                               {'id': 5,
                                'is_visible': '1',
                                'x': 448,
                                'y': 1003},
                               {'id': 6, 'is_visible': '1', 'x': 774, 'y

In [221]:

n=5


In [271]:
keypoints.video_list[0,0]

array(['-08Vnk8XONY'],
      dtype='<U11')

In [227]:
keypoints.annolist[0,n].annorect[0,2].annopoints[0,0].point[0,0].is_visible

array(['1'],
      dtype='<U1')

In [19]:
keypoints.dtype

dtype([('image', 'O'), ('annorect', 'O'), ('frame_sec', 'O'), ('vididx', 'O')])

In [32]:
keypoints[0]['image']['name'][0][0][0]

'037454012.jpg'

In [64]:
n=1
keypoints[1]['annorect']['objpos'][0][n]['x']


array([[array([[338]], dtype=uint16)]], dtype=object)

In [58]:
keypoints[1]['annorect']['scale']

array([[array([[ 8.07816613]]), array([[ 8.90412938]])]], dtype=object)

In [27]:
keypoints[0]['frame_sec']

array([], shape=(1, 0), dtype=float64)

In [28]:
keypoints[0]['vididx']

array([], shape=(1, 0), dtype=float64)